In [ ]:
input_folder = '/Users/dp/Downloads/weili/*/'
output_folder = '/Users/dp/Desktop/missense_microscope/weili/'

num_to_prot = {
    '1-1': 'Dicer WT',
    '1-2': 'Dicer mut',
    '1-3': 'RQCD1 WT',
    '1-4': 'RQCD1 mut',
    '1-5': 'CNOT1 WT',
    '1-6': 'CNOT1 mut',
    '1-7': 'DDX3X WT',
    '1-8': 'DDX3X mut',
    }
def _mk(_):
    if not os.path.exists(_):
        os.system(f'mkdir {_}')
        
_mk(output_folder)
for fname in glob.glob(input_folder + '/*tif'):
    dirname = os.path.dirname(fname)
    bname = os.path.basename(fname)
    
    new_name = re.sub('c2', 'GFP', bname)
    new_name = re.sub('c3', 'DAPI', bname)
    for k in num_to_prot:
        pat = k
        if pat in new_name[:3]:
            new_name = re.sub(pat, num_to_prot[k], new_name)
            break
    new_name = dirname + '/' + new_name
    
    _mk('/Users/dp/Desktop/missense_microscope/')
    print(f"{fname} > {new_name}")


In [ ]:
import numpy as np
import cv2
import os, re, sys, glob

from matplotlib import pyplot as plt


def merge_file(path_green, path_blue, gene_name='', outdir=None):
    if outdir is None:
        outdir = os.path.dirname(path_green)
       
    green = cv2.imread(path_green)
    blue = cv2.imread(path_blue)
    b1,g1,r1 = cv2.split(green)
    b2,g2,r2 = cv2.split(blue)
    img = cv2.merge((r2,g1,b2))
    #plt.imsave(arr=img, fname=f'{outdir}/{gene_name}_Composite.tif')
    plt.imsave(arr=img, fname=f'{outdir}/{gene_name}_Composite.jpg')
    print('Wrote to ' + f'{outdir}/{gene_name}_Composite.jpg')
    plt.clf()
    plt.close()
    

def seems_green(name):
    _ = os.path.basename(name)
    if ('Alexa' in _) or ('GFP' in _) or ('c3' in _):
        return True
    return False


def seems_blue(name):
    _ = os.path.basename(name)
    if re.search('DAPI', os.path.basename(_), re.IGNORECASE) or ('c4' in _):
        return True
    return False    


processed_dirs = set()
for fname in glob.glob('./edit/*/*/*TIF'):
   
    gene_name = re.search('edit/([^/]+)/', fname).group(1)
    dirname = os.path.dirname(fname)
    if dirname in processed_dirs:
        continue
    #print(dirname)
    fnames = glob.glob(dirname + '/*')
    merged = [x for x in fnames if '+' in os.path.basename(x)]
    dapi = [x for x in fnames if seems_blue(x) and (x not in merged)]
    green = [x for x in fnames if seems_green(x) and (x not in merged)]
    if len(dapi) != 1 or len(green) != 1:
        print(f"Confused with files: {fnames}")
        continue
    dapi = dapi[0]
    green = green[0]
    #print(dapi)
    #print(',,,', green)
    #print(fnames)
    merge_file(
        green, dapi,
        #'edit/a1cf/190730_40X-0001/190730_40X-0001_c3.TIF',
        #'edit/a1cf/190730_40X-0001/190730_40X-0001_c4.TIF',
        gene_name=gene_name)
    processed_dirs.add(dirname)

In [ ]:
import os
def mkdir(_p):
    if not os.path.exists(_p):
        os.system(f'mkdir {_p}')

#def cp(fname):
#    if os.path.isdir(fname):
def proclaim(_):
    print(_)
    os.system(_)

def rm_gz(input_dir):
    for (dirpath, dirnames, filenames) in os.walk(input_dir):
        #print(dirpath)#, filenames)
        to_rm = [x for x in filenames if re.search('\.gz', x)]
        #print(to_rm)
        for fname in to_rm:
            proclaim("rm \"" + dirpath + "/" + fname + '"')
#        tifs = [x for x in filenames if re.search('\.TIF$', x)]
#        if len(tifs):
#rm_gz('/Users/dp/Desktop/missense/edit/')

In [ ]:
def convert_tif_jpeg(input_dir):
    outdir = f"{input_dir}/jpegs/"
    mkdir(outdir)
    #for (dirpath, dirnames, filenames) in os.walk(input_dir):
    for fname in glob.glob(f'{input_dir}/*TIF') + glob.glob(f'{input_dir}/*tif') + glob.glob(f'{input_dir}/*jpg'):
        #print(fname)
        img = cv2.imread(fname)
        outname = ".".join(os.path.basename(fname).split('.')[:-1]) + ".jpg"
        #print(f'{outdir}/{outname}')
        plt.imsave(arr=img, fname=f'{outdir}/{outname}')
        plt.clf()
        plt.close()
        
        
def put_three_jpg_in_grid(fname1, fname2, fname3):
    img1 = cv2.imread(fname1)
    img2 = cv2.imread(fname2)
    img3 = cv2.imread(fname3)
    
convert_tif_jpeg('/Users/dp/Desktop/missense_microscope/')

In [ ]:
def center(img, verbose=False):
    if verbose:
        print(img.shape)
    width = 1800
    height = 1800
    excess_rows = img.shape[0] - height
    
    if excess_rows > 1:
        if verbose:
            print(f"Removing {excess_rows}")
        center_top = int(excess_rows/2)
        center_bottom = img.shape[0] - center_top
        img = img[center_top:center_bottom]
        
    excess_cols = img.shape[1] - width
    
    if excess_cols > 1:
        if verbose:
            print(f"Removing cols {excess_cols}")
        center_left = int(excess_cols/2)
        center_right = img.shape[1] - center_left
        img = img[:, center_left:center_right]
        
    return img

def put_three_jpg_in_grid(fname, axes, y_pos=0, title=''):
    gene = re.search('Image_(.*)_Compos', fname)
    if gene:
        title = gene.group(1)
    print(gene.group(1))
    fnameB = re.sub('Image_', 'Image_B', fname)
    fnameG = re.sub('Image_', 'Image_G', fname)
    
    if not os.path.exists(fnameB) or not os.path.exists(fnameG):
        return
    
    img = cv2.imread(fname)
    imgB = cv2.imread(fnameB)
    imgG = cv2.imread(fnameG)
    
    img = center(img, verbose=True)
    imgB = center(imgB)
    imgG = center(imgG)
    
    ax[y_pos][0].imshow(img)
    ax[y_pos][0].set_ylabel(title)
    ax[y_pos][1].imshow(imgB)
    ax[y_pos][2].imshow(imgG)
    for _ax_arr in ax:
        for _ax in _ax_arr:
            _ax.set_xticks([])
            _ax.set_yticks([])
    return ax



fname_list = [
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_a1cf_Composite.jpg_0000003_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_a1cfmut_Composite.jpg_0000002_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_crnkl1_Composite.jpg_0000006_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_crnkl1_mut_Composite.jpg_0000007_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_dcp1b_Composite.jpg_0000008_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_dcp1b_mut_Composite.jpg_0000009_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_ddx50_Composite.jpg_0000013_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_ddx50mut_Composite.jpg_0000014_02.jpg',
    # Page break
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_eef1b2_Composite.jpg_0000026_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_eef1b2_mut_Composite.jpg_0000027_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_eif1ax_Composite.jpg_0000017_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_eif1ax_mut_Composite.jpg_0000024_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_fubp1_Composite.jpg_0000018_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_fubp1mut_Composite.jpg_0000019_02.jpg',
    
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_hnrnpcl1_Composite.jpg_0000020_03.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_hnrnpcl1_mut_Composite.jpg_0000028_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_khdrbs2_Composite.jpg_0000021_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_khdrbs2_mut_Composite.jpg_0000031_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_kpnb1_Composite.jpg_0000032_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_kpnb1mut_Composite.jpg_0000033_02.jpg',
    
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_nova1_Composite.jpg_0000034_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_nova1mut_Composite.jpg_0000035_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_nufip_Composite.jpg_0000037_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_nufip_mut_Composite.jpg_0000038_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_pabpcl4_Composite.jpg_0000039_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_pabpcl4_mut_Composite.jpg_0000040_02.jpg',
    
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rbfox1_Composite.jpg_0000046_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rbfox1mut_Composite.jpg_0000049_03.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rbm11_Composite.jpg_0000050_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rbm11mut_Composite.jpg_0000051_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rbm39_Composite.jpg_0000053_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rbm39mut_Composite.jpg_0000054_03.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_sf3b1_Composite.jpg_0000057_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_sf3b1_625c_Composite.jpg_0000059_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_sf3b1',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_sf3b1_k700e_Composite.jpg_0000063_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_sf3b1_902_Composite.jpg_0000061_02.jpg',
#    '/Users/dp/Desktop/missense_microscope/jpegs/Image_',
    
]

print(f"Input filenames (constructs): {len(fname_list)}")
def chunks(_list, chunk_len):
    for i in range(0, len(_list), chunk_len):
        yield _list[i:i + chunk_len]
        
def write_a_page(fnames_on_page, fig, ax):
    
    for y_pos, fname in enumerate(fnames_on_page):
        ax = put_three_jpg_in_grid(
            fname,
            ax, y_pos=y_pos)
    fig.set_figheight(11)
    fig.set_figwidth(8.5)
    plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0.1, hspace=0.1)
    return ax

mkdir('figs/')
for n, figset in enumerate(chunks(fname_list, 6)):
    #if n == 0:
    #    continue
    first_gene = re.search('Image_(.*)_Compos', figset[0]).group(1)
    last_gene = re.search('Image_(.*)_Compos', figset[-1]).group(1)
    print('::::', first_gene, 'aAA', last_gene)
    fig, ax = plt.subplots(len(figset), 3)
    print(figset)
    ax = write_a_page(figset, fig, ax)
    fig.savefig(f'./figs/{first_gene}_to_{last_gene}.pdf')
    plt.show()
    plt.clf()
    plt.close()


In [ ]:
def center(img, verbose=False):
    if verbose:
        print(img.shape)
    width = 1800
    height = 1800
    excess_rows = img.shape[0] - height
    
    if excess_rows > 1:
        if verbose:
            print(f"Removing {excess_rows}")
        center_top = int(excess_rows/2)
        center_bottom = img.shape[0] - center_top
        img = img[center_top:center_bottom]
        
    excess_cols = img.shape[1] - width
    
    if excess_cols > 1:
        if verbose:
            print(f"Removing cols {excess_cols}")
        center_left = int(excess_cols/2)
        center_right = img.shape[1] - center_left
        img = img[:, center_left:center_right]
        
    return img

def put_three_jpg_in_grid(fname, axes, y_pos=0, x_pos=0, title=''):
    gene = re.search('Image_(.*)_Compos', fname)
    if gene:
        title = gene.group(1)
    print(gene.group(1))
    fnameB = re.sub('Image_', 'Image_B', fname)
    fnameG = re.sub('Image_', 'Image_G', fname)
    
    if not os.path.exists(fnameB) or not os.path.exists(fnameG):
        return
    
    img = cv2.imread(fname)
    imgB = cv2.imread(fnameB)
    imgG = cv2.imread(fnameG)
    
    img = center(img, verbose=True)
    imgB = center(imgB)
    imgG = center(imgG)
    
    ax[y_pos][x_pos].imshow(img)
    ax[y_pos][x_pos].set_ylabel(title)
    ax[y_pos][x_pos + 1].imshow(imgB)
    ax[y_pos][x_pos + 2].imshow(imgG)
    
    for _ax_arr in ax:
        for _ax in _ax_arr:
            _ax.set_xticks([])
            _ax.set_yticks([])
            
    return ax


fname_list = [
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_a1cf_Composite.jpg_0000003_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_a1cfmut_Composite.jpg_0000002_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_crnkl1_Composite.jpg_0000006_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_crnkl1_mut_Composite.jpg_0000007_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_dcp1b_Composite.jpg_0000008_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_dcp1b_mut_Composite.jpg_0000009_02.jpg',
    # Page break
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_ddx50_Composite.jpg_0000013_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_ddx50mut_Composite.jpg_0000014_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_eef1b2_Composite.jpg_0000026_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_eef1b2_mut_Composite.jpg_0000027_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_eif1ax_Composite.jpg_0000017_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_eif1ax_mut_Composite.jpg_0000024_02.jpg',

    '/Users/dp/Desktop/missense_microscope/jpegs/Image_fubp1_Composite.jpg_0000018_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_fubp1mut_Composite.jpg_0000019_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_hnrnpcl1_Composite.jpg_0000020_03.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_hnrnpcl1_mut_Composite.jpg_0000028_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_khdrbs2_Composite.jpg_0000021_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_khdrbs2_mut_Composite.jpg_0000031_02.jpg',

    '/Users/dp/Desktop/missense_microscope/jpegs/Image_kpnb1_Composite.jpg_0000032_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_kpnb1mut_Composite.jpg_0000033_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_nova1_Composite.jpg_0000034_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_nova1mut_Composite.jpg_0000035_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_nufip_Composite.jpg_0000037_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_nufip_mut_Composite.jpg_0000038_02.jpg',
    
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_pabpcl4_Composite.jpg_0000039_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_pabpcl4_mut_Composite.jpg_0000040_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rars2_Composite.jpg_0000042_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rars2_mut_Composite.jpg_0000044_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rbfox1_Composite.jpg_0000046_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rbfox1mut_Composite.jpg_0000049_03.jpg',
    
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rbm11_Composite.jpg_0000050_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rbm11mut_Composite.jpg_0000051_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rbm39_Composite.jpg_0000053_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_rbm39mut_Composite.jpg_0000054_03.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_sf3b1_Composite.jpg_0000057_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_sf3b1_625c_Composite.jpg_0000059_02.jpg',

    '/Users/dp/Desktop/missense_microscope/jpegs/Image_sf3b1_625h_Composite.jpg_0000060_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_sf3b1_k700e_Composite.jpg_0000063_02.jpg',
    '/Users/dp/Desktop/missense_microscope/jpegs/Image_sf3b1_902_Composite.jpg_0000061_02.jpg',
    #'/Users/dp/Desktop/missense_microscope/jpegs/Image_',
    #'/Users/dp/Desktop/missense_microscope/jpegs/Image_',
    #'/Users/dp/Desktop/missense_microscope/jpegs/Image_',
    
    
    
]
print(f"Samples: {len(fname_list)}")
def chunks(_list, chunk_len):
    for i in range(0, len(_list), chunk_len):
        yield _list[i:i + chunk_len]
        
def write_a_page(fnames_on_page, fig, ax):
    
    y_pos = -1
    for n, fname in enumerate(fnames_on_page):
        if n % 2 == 1:
            x_pos = 3
        else:
            x_pos = 0
            y_pos += 1
        ax = put_three_jpg_in_grid(
            fname,
            ax, y_pos=y_pos, x_pos=x_pos)
    fig.set_figheight(11)
    fig.set_figwidth(8.5)
    plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0.1, hspace=0.01)
    return ax

mkdir('figs/')
for figset in chunks(fname_list, 12):
    first_gene = re.search('Image_(.*)_Compos', figset[0]).group(1)
    last_gene = re.search('Image_(.*)_Compos', figset[-1]).group(1)
    
    fig, ax = plt.subplots(int(len(figset)/2), 6)
    print(figset)
    ax = write_a_page(figset, fig, ax)
    fig.savefig(f'./figs/{first_gene}_to_{last_gene}.pdf')
    plt.show()
    plt.clf()
    plt.close()
